In [13]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI


In [9]:
from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
  name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

In [ ]:
import json
data = json.load(open("reviews.json"))
data["reviews"]

In [17]:
processed_data = []
client = OpenAI()

for rev in data["reviews"]:
  response = client.embeddings.create(
    input=rev["review"],
    model="text-embedding-3-small"
  )
  embedding = response.data[0].embedding
  processed_data.append({
    "values": embedding,
    "id": rev["professor"],
    "metadata": {
      "review": rev["review"],
      "subject": rev["subject"],
      "stars": rev["stars"]
    }
  })

In [ ]:
processed_data[0]

In [19]:
index = pc.Index('rag')
index.upsert(
  vectors=processed_data,
  namespace="ns1"
)

{'upserted_count': 21}

In [20]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 21}},
 'total_vector_count': 21}